In [27]:
#load "Paket.fsx"
Paket.Package ["Newtonsoft.Json";]

#r @"/IfSharp/bin/packages/Newtonsoft.Json/lib/net40/Newtonsoft.Json.dll"


<null>

In [28]:
open Newtonsoft.Json

let jsonify o = JsonConvert.SerializeObject(o, Newtonsoft.Json.Formatting.Indented)
let jout o = jsonify o |> Display

In [56]:
// Day 1
open System
open System.Text.RegularExpressions

let input = """R4, R3, R5, L3, L5, R2, L2, R5, L2, R5, R5, R5, R1, R3, L2, L2, L1, R5, L3, R1, L2, R1, L3, L5, L1, R3, L4, R2, R4, L3, L1, R4, L4, R3, L5, L3, R188, R4, L1, R48, L5, R4, R71, R3, L2, R188, L3, R2, L3, R3, L5, L1, R1, L2, L4, L2, R5, L3, R3, R3, R4, L3, L4, R5, L4, L4, R3, R4, L4, R1, L3, L1, L1, R4, R1, L4, R1, L1, L3, R2, L2, R2, L1, R5, R3, R4, L5, R2, R5, L5, R1, R2, L1, L3, R3, R1, R3, L4, R4, L4, L1, R1, L2, L2, L4, R1, L3, R4, L2, R3, L1, L5, R4, R5, R2, R5, R1, R5, R1, R3, L3, L2, L2, L5, R2, L2, R5, R5, L2, R3, L5, R5, L2, R4, R2, L1, R3, L5, R3, R2, R5, L1, R3, L2, R2, R1"""

type Turn = Right | Left
type Instruction = Turn of Turn | Move
type State = { pos: int*int; facing: int*int; visited: Set<int*int> }
let startState = { pos = (0,0); facing = (0,-1); visited = Set.empty } 
let rotate (x,y) = function | Right -> (y*(-1),x) | Left -> (y,x*(-1)) 
let getDistance (x,y) = abs(x) + abs(y)
let instructions = Regex.Matches(input, "([LR])|(\\d+)") 
                    |> Seq.cast<Match> |> Seq.map (fun m -> m.Value)
                    |> Seq.collect (function | "L" -> [Turn Left] | "R" -> [Turn Right] | n -> [for _ in 1..int(n) -> Move])

let move state instruction = match instruction with 
                                | Turn direction -> { state with facing = rotate state.facing direction }
                                | Move ->
                                    let addv (x,y) (i,j) = (x+i,y+j)
                                    { state with pos = addv state.pos state.facing; visited = Set.add state.pos state.visited  }

let endState = instructions |> Seq.fold move startState

Display (getDistance endState.pos)

let visitedTwice = instructions 
                    |> Seq.scan move startState
                    |> Seq.find (fun f -> f.visited.Contains(f.pos))

Display (getDistance visitedTwice.pos)



271

153

/notebooks/input.fsx(18,53): warning FS0667: The field labels and expected type of this record expression or pattern do not uniquely determine a corresponding record type

In [34]:
module DayTwo =
    let solve (keypad:string[]) startPos input =
        let lookup (x,y) = keypad.[y].[x]
        let isValid pos = lookup pos <> ' '
        let followInstruction pos instruction = 
            let addv (x,y) (i,j) = x+i,y+j
            let move = match instruction with | 'U' -> (0,-1) | 'D' -> (0,1) | 'R' -> (1,0) | 'L' -> (-1,0) | _ -> (0,0)
            let newPos = addv pos move
            if isValid newPos then newPos else pos
        let followLine = Seq.fold followInstruction
        input 
            |> Seq.scan followLine startPos
            |> Seq.skip 1 
            |> Seq.map (lookup >> string) 
            |> System.String.Concat 

    let keypadA = [| "     "; " 123 "; " 456 "; " 789 "; "     " |]
    let keypadB = [| "       "; "   1   "; "  234  "; " 56789 "; "  ABC  "; "   D   "; "       " |]

    let input = """LRRLLLRDRURUDLRDDURULRULLDLRRLRLDULUDDDDLLRRLDUUDULDRURRLDULRRULDLRDUDLRLLLULDUURRRRURURULURRULRURDLULURDRDURDRLRRUUDRULLLLLDRULDDLLRDLURRLDUURDLRLUDLDUDLURLRLDRLUDUULRRRUUULLRDURUDRUDRDRLLDLDDDLDLRRULDUUDULRUDDRLLURDDRLDDUDLLLLULRDDUDDUUULRULUULRLLDULUDLLLLURRLDLUDLDDLDRLRRDRDUDDDLLLLLRRLLRLUDLULLDLDDRRUDDRLRDDURRDULLLURLRDLRRLRDLDURLDDULLLDRRURDULUDUDLLLDDDLLRLDDDLLRRLLURUULULDDDUDULUUURRUUDLDULULDRDDLURURDLDLULDUDUDDDDD
    RUURUDRDUULRDDLRLLLULLDDUDRDURDLRUULLLLUDUDRRUDUULRRUUDDURDDDLLLLRRUURULULLUDDLRDUDULRURRDRDLDLDUULUULUDDLUDRLULRUDRDDDLRRUUDRRLULUULDULDDLRLURDRLURRRRULDDRLDLLLRULLDURRLUDULDRDUDRLRLULRURDDRLUDLRURDDRDULUDLDLLLDRLRUDLLLLLDUDRDUURUDDUDLDLDUDLLDLRRDLULLURLDDUDDRDUDLDDUULDRLURRDLDLLUUDLDLURRLDRDDLLDLRLULUDRDLLLDRLRLLLDRUULUDLLURDLLUURUDURDDRDRDDUDDRRLLUULRRDRULRURRULLDDDUDULDDRULRLDURLUDULDLDDDLRULLULULUDLDDRDLRDRDLDULRRLRLRLLLLLDDDRDDULRDULRRLDLUDDDDLUDRLLDLURDLRDLDRDRDURRDUDULLLDLUDLDRLRRDDDRRLRLLULDRLRLLLLDUUURDLLULLUDDRLULRDLDLDURRRUURDUDRDLLLLLLDDDURLDULDRLLDUDRULRRDLDUDRLLUUUDULURRUR
    URRRLRLLDDDRRLDLDLUDRDRDLDUDDDLDRRDRLDULRRDRRDUDRRUUDUUUDLLUURLRDRRURRRRUDRLLLLRRDULRDDRUDLRLUDURRLRLDDRRLUULURLURURUDRULDUUDLULUURRRDDLRDLUDRDLDDDLRUDURRLLRDDRDRLRLLRLRUUDRRLDLUDRURUULDUURDRUULDLLDRDLRDUUDLRLRRLUDRRUULRDDRDLDDULRRRURLRDDRLLLRDRLURDLDRUULDRRRLURURUUUULULRURULRLDDDDLULRLRULDUDDULRUULRRRRRLRLRUDDURLDRRDDULLUULLDLUDDDUURLRRLDULUUDDULDDUULLLRUDLLLRDDDLUUURLDUDRLLLDRRLDDLUDLLDLRRRLDDRUULULUURDDLUR
    UULDRLUULURDRLDULURLUDULDRRDULULUDLLDURRRURDRLRLLRLDDLURRDLUUDLULRDULDRDLULULULDDLURULLULUDDRRULULULRDULRUURRRUDLRLURDRURDRRUDLDDUURDUUDLULDUDDLUUURURLRRDLULURDURRRURURDUURDRRURRDDULRULRRDRRDRUUUUULRLUUUDUUULLRRDRDULRDDULDRRULRLDLLULUUULUUDRDUUUDLLULDDRRDULUURRDUULLUUDRLLDUDLLLURURLUDDLRURRDRLDDURLDLLUURLDUURULLLRURURLULLLUURUUULLDLRDLUDDRRDDUUDLRURDDDRURUURURRRDLUDRLUULDUDLRUUDRLDRRDLDLDLRUDDDDRRDLDDDLLDLULLRUDDUDDDLDDUURLDUDLRDRURULDULULUDRRDLLRURDULDDRRDLUURUUULULRURDUUDLULLURUDDRLDDUDURRDURRUURLDLLDDUUDLLUURDRULLRRUUURRLLDRRDLURRURDULDDDDRDD
    LLRUDRUUDUDLRDRDRRLRDRRUDRDURURRLDDDDLRDURDLRRUDRLLRDDUULRULURRRLRULDUURLRURLRLDUDLLDULULDUUURLRURUDDDDRDDLLURDLDRRUDRLDULLRULULLRURRLLURDLLLRRRRDRULRUDUDUDULUURUUURDDLDRDRUUURLDRULDUDULRLRLULLDURRRRURRRDRULULUDLULDDRLRRULLDURUDDUULRUUURDRRLULRRDLDUDURUUUUUURRUUULURDUUDLLUURDLULUDDLUUULLDURLDRRDDLRRRDRLLDRRLUDRLLLDRUULDUDRDDRDRRRLUDUDRRRLDRLRURDLRULRDUUDRRLLRLUUUUURRURLURDRRUURDRRLULUDULRLLURDLLULDDDLRDULLLUDRLURDDLRURLLRDRDULULDDRDDLDDRUUURDUUUDURRLRDUDLRRLRRRDUULDRDUDRLDLRULDL"""

    let inputlines = input.Split [|'\n'|]

Display <| DayTwo.solve DayTwo.keypadA (2,2) DayTwo.inputlines
Display <| DayTwo.solve DayTwo.keypadB (1,3) DayTwo.inputlines

"73597"

"A47DA"

<null>

In [33]:
open System
open System.IO

module Triangle =
    let validSides s1 s2 s3 = ((s1 + s2) > s3) && ((s2 + s3) > s1) && ((s1 + s3) > s2)
    let isValid (sides:int[]) = 
        let a = sides.[0]
        let b = sides.[1]
        let c = sides.[2]
        validSides a b c

module DayThree =
    let get_inputs () = File.ReadAllLines("/notebooks/aoc/day3_inputs") |> Array.map (fun s -> s.Trim())
                         |> Array.map (fun s -> s.Split [| ' ' |] 
                                                 |> Array.filter (fun n -> n.Length > 0)
                                                 |> Array.map Int32.Parse)                                                 
    let pivotFlatten rows = rows
                            |> Array.collect (fun r -> r |> Array.mapi (fun idx el -> (idx,el)))
                            |> Array.groupBy fst
                            |> Array.collect (fun (idx, t) -> t |> Array.map snd)                    
    let chunk s = s |> Array.chunkBySize 3        
    
    let possibleTriangleRows = get_inputs() |> Array.filter Triangle.isValid
    let possibleTriangleColumns = get_inputs() |> pivotFlatten |> chunk |> Array.filter Triangle.isValid
                     
DayThree.possibleTriangleRows |> Array.length |> Display
DayThree.possibleTriangleColumns |> Array.length |> Display


982

1826

<null>

In [45]:
open System
open System.IO
open System.Text.RegularExpressions

module DayFour =
    module Cipher =
        let chars = ['a' .. 'z']
        let shift cnt s = 
            let rot c =
                match c with
                | c when c > 64 && c < 91 -> ((c - 65 + cnt) % 26) + 65
                | c when c > 96 && c < 123 -> ((c - 97 + cnt) % 26) + 97
                | _ -> c
            s |> Array.ofSeq
            |> Array.map(int >> rot >> char)
            |> (fun seq -> new string(seq))

            

    type RoomIdentifier = {
        Name: string;
        Id: int;
        Checksum: string;
    }
    
    let sanitize (name:string) =
        name.Replace("-", "")

    let parse_room input =
        let m = Regex.Match(input, "(?<name>[a-z|-]+)-{1}(?<id>\d+)\[(?<checksum>[a-z]{5})\]")
        match m.Success with
        | true -> Some { Name=m.Groups.["name"].Value; Id=(Int32.Parse m.Groups.["id"].Value); Checksum=m.Groups.["checksum"].Value }
        | _ -> None
        
    let count_letters : string -> (char * int) seq = Seq.countBy id
    let top_five_letters (s:string) =
        sanitize s
        |> count_letters
        |> Seq.sortBy (fun (letter, cnt) -> (cnt * -1), letter)
        |> Seq.take 5
        |> Seq.map (fun (letter, cnt) -> letter)
        |> String.Concat
        
    let room_is_valid room = 
        (top_five_letters room.Name) = room.Checksum
    

    let get_inputs () =
        File.ReadAllLines "/notebooks/aoc/day4_inputs"
                |> Seq.ofArray
                |> Seq.map parse_room
                
    let compute_solution inputs =
        inputs
            |> Seq.filter (fun r -> match r with
                                    | Some ri -> true
                                    | None -> false)
            |> Seq.map (fun r -> r.Value)
            |> Seq.filter room_is_valid
            |> Seq.fold (fun sum (r:RoomIdentifier) -> sum + r.Id) 0
    
    let decryptRoom r =
        {
            Name=sanitize r.Name |> Cipher.shift r.Id ;
            Id=r.Id;
            Checksum=r.Checksum;
        }
    
    let decrypt_list inputs =
        inputs 
           |> Seq.filter (fun r -> match r with
                                    | Some ri -> true
                                    | None -> false)
           |> Seq.map (fun r -> r.Value)
           |> Seq.map decryptRoom
           |> Seq.filter (fun s -> s.Name.Contains "north")
            

//DayFour.get_inputs() |> DayFour.compute_solution

DayFour.get_inputs() |> DayFour.decrypt_list |> jout


"[
  {
    "Name": "northpoleobjectstorage",
    "Id": 548,
    "Checksum": "mcrpa"
  }
]"

<null>

In [ ]:
open System.Security.Cryptography
open System.Text
open System

module String =
    let char i (s:string) = s.Chars i
    let lower (s:string) = s.ToLower()

module DayFive =
    let hasher = System.Security.Cryptography.MD5.Create()

    let md5 (input:string) =
        input
        |> (System.Text.Encoding.ASCII.GetBytes >> hasher.ComputeHash)
        
    let hashstring (h:byte[]) = 
        h
        |> Seq.map (fun c -> c.ToString("X2"))
        |> Seq.reduce (+)
        
    let hashes door = 
        let isSignificant (hash:byte[]) = 
            hash.[0] = 0uy && hash.[1] = 0uy && hash.[2] <= 0xFuy
        door |> sprintf "%s%d" |> Seq.initInfinite
             |> Seq.map md5
             |> Seq.filter isSignificant
             |> Seq.map hashstring
             |> Seq.map (fun h ->
                             Display h
                             h)

    let computeFirstCode door =
        hashes door |> Seq.take 8
                    |> Seq.map (String.char 5)
                    |> String.Concat
                    |> String.lower
                    
    let subChar (state:string) (idx, ch) =
        if idx < 8 then match state.[idx] with
                        | '*' -> 
                            let s = sprintf "%s%c%s" (state.Substring(0,idx)) ch (state.Substring(idx+1))
                            Display s
                            s
                        | _ -> state
        else state
    
    let computeSecondCode door =
        let initState = (String.replicate (String.length door) "*")
        hashes door |> Seq.map (fun h -> int h.[5] - int '0', h.[6])
                    |> Seq.fold subChar initState
                    |> String.lower
                    
//Display (DayFive.computeFirstCode "cxdnnyjw")
Display (DayFive.computeSecondCode "cxdnnyjw")
    

"00000FE1E92080B9951B053E70E31FCB"

"000007C827126C81FA664211693F2540"

"*******C"

"000007880153F1B804481A39A6D2E86A"

"00000A6E225253B6AAA8F20EFBAAD8B5"

...

"000004FB5C504E08E9EE2F9A679ADC29"

"00000B10F56C243B8498B620D2386C73"

"000007E2BE8A8305A2A880F656D5DAF7"

"00000F814E9461D1561AC7F3DA140E74"

"00000F206761DF519060C8B4ADBB3F08"

In [45]:
open System
open System.IO

module DaySix =
    let input = File.ReadAllLines("/notebooks/aoc/day6_inputs") |> Seq.map (fun s -> s.ToCharArray())
    let messageLength = input |> Seq.head |> Array.length
    let countCharacters s = s |> Seq.countBy id |> Seq.sortByDescending (fun (k,v) -> v)
    let countPosition lines position = lines |> Seq.map (Array.item position) |> countCharacters
    let solvePosition sorter position = position |> countPosition input |> sorter |> (fun (k,v) -> k)
    
    let solvePartOne = solvePosition Seq.head
    let solvePartTwo = solvePosition Seq.last
    
    let getMessage (solver: int->char) = seq {0..(messageLength - 1)} |> Seq.map solver |> String.Concat
    
    let messageOne = getMessage solvePartOne
    let messageTwo = getMessage solvePartTwo

Display DaySix.messageOne
Display DaySix.messageTwo



"cyxeoccr"

"batwpask"

<null>

In [19]:
open System
open System.IO
open System.Text.RegularExpressions

module DaySeven =
    let input = File.ReadAllLines "/notebooks/aoc/day7_inputs"
    
    let filterByIndex predicate sequence = 
        sequence |> Seq.indexed |> Seq.filter (fst >> predicate) |> Seq.map snd 

    let parseIpAddress ipAddress =
        let parts = Regex.Split(ipAddress,@"\[(.*?)\]")
        let supernetSequences = parts |> filterByIndex (fun n -> n % 2= 0) |> Seq.toArray 
        let hypernetSequences = parts |> filterByIndex (fun n -> n % 2= 1) |> Seq.toArray
        supernetSequences, hypernetSequences

    let supportsTls ipAddress = 
        let super,hyper = parseIpAddress ipAddress
        let containsAbba s = s |> Seq.windowed 4 |> Seq.exists (function | [|a;b;c;d|] -> a=d&&b=c&&a<>b | _ -> false)
        (super |> Array.exists containsAbba) && not (hyper |> Array.exists containsAbba)

    let partA = input |> Seq.filter supportsTls |> Seq.length

    let supportsSsl ipAddress = 
        let super,hyper = parseIpAddress ipAddress
        let findAbas s = s |> Seq.windowed 3 |> Seq.filter (function | [|a;b;c|] -> a=c&&a<>b | _ -> false) |> Seq.map System.String
        let abas = super |> Seq.collect findAbas
        let makeBab (aba:string) = sprintf "%c%c%c" aba.[1] aba.[0] aba.[1]
        let babExists bab = hyper |> Seq.exists (fun s -> s.Contains(bab))
        super |> Seq.collect findAbas |> Seq.exists (makeBab >> babExists)

    let partB = input |> Seq.filter supportsSsl |> Seq.length
                    
Display DaySeven.partA
Display DaySeven.partB

105

258

<null>

In [54]:
open System
open System.IO
open System.Text.RegularExpressions

module DayEight =
    let (|Regex|_|) pattern input =
        let m = Regex.Match(input, pattern)
        if m.Success then Some(List.tail [ for g in m.Groups -> g.Value ]) else None


    
    let initW,initH = 50,6
    let initialScreen = Array2D.init initW initH (fun _ _ -> false)
    
    let createRectangle (w,h) = for x in [0..(w-1)] do for y in [0..h-1] do initialScreen.[x, y] <- true
    let rotate_x y dist =
        let row = [0..initW-1] |> List.map (fun x -> initialScreen.[x, y])
        for x in [0..initW-1] do initialScreen.[(x + dist) % initW, y] <- row.[x]
    let rotate_y x dist =
        let column = [0..initH-1] |> List.map (fun y -> initialScreen.[x, y])
        for y in [0..initH-1] do initialScreen.[x, (y + dist) % initH] <- column.[y]
        
    let process_instruction line =
        match line with
         | Regex @"rect (\d+)x(\d+)" [w; h] -> createRectangle (Int32.Parse w,Int32.Parse h)
         | Regex @"rotate row y=(\d+) by (\d+)" [y; dist] -> rotate_x (Int32.Parse y) (Int32.Parse dist)
         | Regex @"rotate column x=(\d+) by (\d+)" [x; dist] -> rotate_y (Int32.Parse x) (Int32.Parse dist)
         | _ -> failwith ("Unknown instruction for " + line)
         
    let input = File.ReadAllLines "/notebooks/aoc/day8_inputs" 
                        |> List.ofSeq |> List.iter process_instruction
                        
    let printScreen (screen: bool[,]) =        
        for y in [0..initH-1] do
            let mutable s = ""
            for x in [0..initW-1] do 
                s <- s + (if screen.[x, y] then "x" else " ")
            Display s



Display (DayEight.initialScreen |> Seq.cast<bool> |> Seq.filter id |> Seq.length |> (sprintf "Part 1: %d"))
(DayEight.printScreen DayEight.initialScreen) |> Display


"Part 1: 123"

" xx  xxxx xxx  x  x xxx  xxxx xxx    xx xxx   xxx "

"x  x x    x  x x  x x  x    x x  x    x x  x x    "

"x  x xxx  xxx  x  x x  x   x  xxx     x x  x x    "

"xxxx x    x  x x  x xxx   x   x  x    x xxx   xx  "

"x  x x    x  x x  x x    x    x  x x  x x       x "

"x  x x    xxx   xx  x    xxxx xxx   xx  x    xxx  "

<null>

In [74]:
open System
open System.IO
open System.Text.RegularExpressions

let (|Regex|_|) pattern input =
        let m = Regex.Match(input, pattern)
        if m.Success then Some(List.tail [ for g in m.Groups -> g.Value ]) else None

module DayNine = 
    let rec expand (input : string) fn =
        let cnt = input |> Seq.takeWhile (fun c -> c <> '(') |> Seq.length
        if cnt = input.Length then bigint cnt else
            let s = input |> Seq.skip (cnt+1) |> Seq.takeWhile (fun c -> c <> ')') |> String.Concat
            let [n; t] = List.tail [for g in (Regex.Match (s, @"(\d+)x(\d+)")).Groups -> g.Value] |> List.map int
            bigint cnt 
                + (bigint t * (fn input (cnt + s.Length + 2) n)) 
                + (expand (input |> Seq.skip (cnt + s.Length + 2 + n) |> String.Concat) fn)


let input = (File.ReadAllLines "/notebooks/aoc/day9") |> Seq.head
Display (sprintf "Part 1:  %A" (DayNine.expand input (fun _ _ nchars -> bigint nchars)))

let rec f (s : string) k n = DayNine.expand (s |> Seq.skip k |> Seq.take n |> String.Concat) f
Display (sprintf "Part 2:  %A" (DayNine.expand input f))
                    
    

"Part 1:  120765"

"Part 2:  11658395076"

/notebooks/input.fsx(14,17): warning FS0025: Incomplete pattern matches on this expression. For example, the value '[_;_;_]' may indicate a case not covered by the pattern(s).